# **Data Visualization Notebook**

## Objectives

* Answer business requirement 1:
    * The client is interested to have a study to visually differentiate a cherry leaf that is healthy from one that contains powdery mildew.

## Inputs

* inputs/mildew_dataset/cherry-leaves/train
* inputs/mildew_dataset/cherry-leaves/test
* inputs/mildew_dataset/cherry-leaves/validation

## Outputs

* Image shape embeddings pickle file
* Mean and variability of images per label plot
* Plot to distinguish contrast between a healthy cherry leaves and those that contain powdery mildew
* Generate code that answers business requirement 1 and can be used to build image montage on Streamlit dashboard


---

# Set working directory

In [1]:
cwd = os.getcwd()

Section 1 content

In [2]:
os.chdir('/workspace/mildewproject')
print("You set a new current directory")

You set a new current directory


In [3]:
work_dir = os.getcwd()
work_dir

'/workspace/mildewproject'

---

# Section 2

Section 2 content